# Human-in-the-loop Machine Learning with GroundWork, Raster Vision, and STAC

<img src="img/groundwork.png" alt="GroundWork" width=200/>

<img src="img/raster-vision-logo.png" alt="Raster Vision" width=300/>

<img src="img/stac.png" alt="STAC" width=300/>

This notebook will walk you through a process in which you get a base model from _somewhere_, use it to create predictions, upload those predictions to the [GroundWork](https://groundwork.azavea.com/) application, correct those predictions, and use the corrections to improve your original model. Once you can get from a model to predictions to an improved model, you can repeat that process any number of times. That's the loop. You're the human. Let's rock.

A note on the environment: this notebook is written to run inside a container launched by `docker/run` on `master` in the `raster-vision` repo. If you're not sure how a dependency is available, why we didn't set things up in advance, or why we're in Python 3.6, that's the reason. Additionally, the environment configured by this repository's ansible scripts makes sure that we have some specific data in a specific location. If you run this outside of that configured environment, you'll need to specificy a different path to your data.

## From predictions to a GroundWork project

### Step 1: Set up dependencies

The `raster-vision` container has lots of things we'll need (PyTorch, the python scientific stack, a bunch of system dependencies) already available, but we'll need a few more dependencies.

These additional dependencies and what we'll use them for are:

- `geopandas`: to figure out which _tasks_ (more below) to put our predictions in
- `shapely`: for transforming geojson to python data
- `rtree`: `geopandas` needs this for spatial joins, but it's an optional dependency, so we have to say we want it

Run the cell below, then restart the notebook (`0 0` or `Kernel` -> `restart`).

In [ ]:
%pip install geopandas shapely rtree seedir

After that, we'll import everything we're going to need over the rest of the notebook:

In [ ]:
from copy import deepcopy
import functools
from itertools import zip_longest
import json
from os.path import join
from random import random
import time
from uuid import uuid4
from pprint import pformat
from copy import deepcopy
from tempfile import TemporaryDirectory

import geopandas as gpd
from geopandas.tools import sjoin
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Polygon as MPolygon
import numpy as np
import requests
from scipy.special import softmax
import shapely
from shapely.geometry import MultiPolygon, Polygon, shape
from shapely.ops import unary_union
import seedir as sd

from rastervision.pytorch_backend.examples.utils import read_stac
from rastervision.pipeline.file_system import unzip

%matplotlib inline

### Step 2: Configuration

You'll configure a few values here. The goals of this configuration are to make it so that you can talk to the Raster Foundry API (the same API that powers GroundWork) from within the notebook.

The values you'll configure are:

- `bearer_token`: you can get this from network tools while logged in to GroundWork. This is a JSON Web Token. To see what it represents and learn more about JSON Web Tokens, you can decode it at [jwt.io](https://jwt.io/)
- `url_base`: this value configures the scheme and host for requests to the Raster Foundry API. All of our requests will start with `app.rasterfoundry.com`
- `source_project_id`: this value is a UUID pointing to a project template. GroundWork has two sort of high level grouping concepts -- a _project_ is a specific image that you'd like to do labeling work in, and a _campaign_ is a group of projects. Since we have predictions over an image, we want to work at the project level in this notebook.

To obtain the JSON web token, open the GroundWork campaign that has been created for you called "Jacksonville," open developer tools (right click in the page and choose "inspect"), switch to the network tab, filter to requests containing the word "random" in their url (type `random` into the search bar), and select the first request with the GET method. Then, inspect the request headers, right click on the authorization header, and choose "Copy". We'll also go through these steps in the workshop.

In [ ]:
# Your token for interacting with the Raster Foundry API
bearer_token = "your-token-here"

# common HTTP headers shared by the requests we're going to make
headers = {"Authorization": f"Bearer {bearer_token}"}

# The base URL for the Raster Foundry API
# This will only be different if you're working with a copy of Raster Foundry that lives somewhere else
url_base = "https://app.rasterfoundry.com"

# UUID of your template project
source_project_id = "your-project-id"

Let's make sure your token is correct -- all users can make a request to `https://app.rasterfoundry.com/api/users/me` with their token to see their user information in the Raster Foundry API. If your token is correctly configured, you'll see some JSON output describing your user. If it's incorrectly configured, you'll see an exception in the cell below:

In [ ]:
user_resp = requests.get(f"{url_base}/api/users/me", headers=headers)
user_resp.raise_for_status()
user_resp.json()

If that failed, a common way that your token can end up misconfigured is if you're using Firefox and dragged to highlight the token's value. Because the header value is quite long, Firefox helpfully (citation needed) truncates it, and you end up with a unicode ellipsis (`...`) in the middle of the value. To correct this, instead right click the header's value and choose `copy`.

### Step 3: Create a copy of your template project

The workflow we'll use in this workshop is that each iteration of the human-in-the-loop workflow will create a new project based on the template that we configured above. The steps to copy a project are:

- fetch the existing project
- create a new JSON document like from fields in that project
- POST that new project to the Raster Foundry API
- fetch all the tasks in the existing project
- add them to the new project by changing their project reference

In [ ]:
# Get the source project
@functools.lru_cache(None)
def fetch_project(project_id):
    return requests.get(join(url_base, "api", "annotation-projects", 
                             project_id), headers=headers).json()

# create a new JSON document from the source project
def make_project_clone_json(source_project, iteration_number=0):
    return {
        # the name is just the source project name with a "_HITL" suffix
        "name": f"""{source_project["name"]} {iteration_number}""",
        "projectType": source_project["projectType"],
        "taskSizePixels": 512,
        "aoi": source_project["aoi"],
        "labelersTeamId": source_project["labelersTeamId"],
        "validatorsTeamId": source_project["validatorsTeamId"],
        "projectId": None,
        "campaignId": source_project["campaignId"],
        "status": source_project["status"],
        "tileLayers": source_project["tileLayers"],
        "labelClassGroups": []
    }

# post the project copy to the Raster Foundry API
def post_project(project_json):
    post_hitl_url = join(url_base, "api","annotation-projects")
    post_hitl = requests.post(post_hitl_url, headers=headers, json=project_json)
    post_hitl.raise_for_status()
    return post_hitl.json()

So we can get the source project by bundling that workflow up with `project_id` and `iteration_number` parameters:

In [ ]:
def clone_project(project_id, iteration_number):
    source_project = fetch_project(project_id)
    new_project = make_project_clone_json(source_project, iteration_number)
    return post_project(new_project)

We can then make our first copy of the template project like so:

In [ ]:
hitl_project = clone_project(source_project_id, 1)
f"""https://groundwork.azavea.com/app/campaign/{hitl_project["campaignId"]}/overview?p=0&f=all"""

You can visit that URL in the GroundWork app and select the project you just created to see that we've created a copy of the project that has the same imagery tile layer, but it doesn't appear to have any tasks.

#### Uploading tasks

GroundWork breaks labeling work into more manageable pieces called _tasks_. A task is a specific window within a larger image. If you look at the overview for the source project, each little box over the imagery is a task.

To fill in tasks, we'll need to:

* grab all of the tasks from the source project
* POST them to the new project

Since the number of tasks in the source project can be huge, we'll work in batches.

In [ ]:
# Grab all of the tasks from the source project
def fetch_tasks(annotation_project_id, url_base):
    template_project_tasks_url = join(url_base,"api/annotation-projects/", annotation_project_id, "tasks")
    tasks = requests.get(template_project_tasks_url, headers=headers).json()
    has_next = tasks["hasNext"]
    next_page = 1
    while has_next:
        new_tasks_url = f"{template_project_tasks_url}?page={next_page}"
        next_tasks = requests.get(new_tasks_url, headers=headers).json()
        tasks["features"] += next_tasks["features"]
        has_next = next_tasks["hasNext"]
        next_page += 1
    return tasks

# break all tasks into manageable chunks
# modified from https://docs.python.org/3/library/itertools.html#itertools-recipes
def grouper(iterable, n):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    x = zip_longest(*args)
    # workaround to remove the fill values in the chunks
    return [[ii for ii in i if ii is not None] for i in x]

# POST the tasks to the new project in groups
def copy_tasks_to_project(source_tasks, project_id):
    tasks_post_url = join(url_base, "api", "annotation-projects", project_id, "tasks")
    chunks = grouper(source_tasks['features'], 1250)
    out = {"type": "FeatureCollection", "features": []}
    for chunk in chunks:
        chunk_tasks = {"features": [], "type": "FeatureCollection"}
        for task in chunk:
            # set the status to unlabeled, no matter what it was before
            task["properties"]["status"] = "UNLABELED"
            # set the annotationProjectId to 
            task["properties"]["annotationProjectId"] = project_id
            chunk_tasks["features"] += [task]
    
        chunk_tasks_response = requests.post(tasks_post_url, headers=headers, json=chunk_tasks)
        # make sure this post request doesn't fail silently
        chunk_tasks_response.raise_for_status()
        out["features"] += chunk_tasks_response.json()["features"]
    return out

# One-shot to grab all the tasks and do the copy
def clone_tasks(from_project_id, to_project_id):
    source_tasks = fetch_tasks(source_project_id, url_base)
    return copy_tasks_to_project(source_tasks, to_project_id)

In [ ]:
hitl_project_tasks = clone_tasks(source_project_id, hitl_project["id"])
len(hitl_project_tasks["features"])

### Step 4: upload labels

Now that we have a project and tasks, we can upload our labels. To upload the labels, we'll need to complete three steps:

* associate each predicted label with a label class
* associate each predicted label with a task
* POST all the labels to GroundWork

Each label has to be associated with a task within a project and with a label class. GroundWork associates labels with classes via UUIDs, while Raster Vision only speaks string names, so we'll need to make sure we can translate between the names and IDs. Fortunately, we can get that translation from the campaign.

In [ ]:
# get a dict to map from label name (from Raster Vision) to label class id (GroundWork / Raster Foundry)
def get_class_map(project):
    campaign_id = project['campaignId']
    get_label_class_url = join(url_base, "api", "campaigns", campaign_id, "label-class-groups")
    label_class_summary = requests.get(get_label_class_url, headers=headers).json()
    return {d['name']: d['id'] for d in label_class_summary[0]['labelClasses']}

Joining labels to tasks is slightly more complex. Our labels in this case are chip classification labels. Those chips won't align perfectly with the task grid that we created. However, both the tasks and the chips happen to be square, so we know that if the centroid of a label is located within a task, that's the most appropriate task for the label. We can do this kind of join with geopandas. We'll also need to track the predictions' original geometries though, so we'll return that in a separate map.

In [ ]:
def associate_tasks(predictions_feature_collection, tasks_feature_collection):
    geom_mapping = {}
    tasks_df = gpd.GeoDataFrame.from_features(tasks_feature_collection["features"], crs="epsg:4326")
    copied = deepcopy(predictions_feature_collection)
    for f in copied["features"]:
        f["properties"]["score"] = np.max(softmax(f["properties"]["scores"]))
        geom = shape(f["geometry"])
        ad_hoc_id = str(uuid4())
        geom_mapping[ad_hoc_id] = geom
        f["properties"]["ad-hoc-id"] = ad_hoc_id
        # find the centroids which we will use for easier joining to task grid
        f["geometry"] = geom.centroid
    return {"original_geometries": geom_mapping,
            "joined": sjoin(
                tasks_df,
                gpd.GeoDataFrame.from_features(copied['features'], crs='EPSG:4326'),
                how="left"
            )}

In [ ]:
rv_output_uri = "../base-predictions.json"

with open(rv_output_uri, "r") as inf:
    predictions_feature_collection = json.load(inf)

labels_with_task_ids = associate_tasks(predictions_feature_collection, hitl_project_tasks)

Finally, we can post these predictions to GroundWork:

In [ ]:
# collect the json needed to post labels from each row in the labels with task ID table
def features_to_label_post_body(group, geom_mapping, class_map):
    def feature_to_label(r):
        try:
            invert = r["class_name"] == "background"
            return { "type": "Feature",
              "properties": {
                "annotationLabelClasses": [class_map["boat"]],
                "score": r["score"] if not invert else 1 - r["score"]
              },
              "geometry": shapely.geometry.mapping(geom_mapping[r["ad-hoc-id"]]),
             "id": r["id"]
            }
        except:
            print("Failing row:")
            print(r)
            raise

    return {
      "type":"FeatureCollection",
      "features": [feature_to_label(r) for _, r in group.iterrows() if not gpd.pd.isna(r['class_name'])],
        "nextStatus":"LABELED"
    }

def upload_labels(project_id, joined, original_geometries, class_map):
    for task_id, task_labels in joined.groupby("id"):
        label_upload_body = features_to_label_post_body(task_labels, original_geometries, class_map)
        label_upload_url = join(url_base, "api", "annotation-projects", project_id, "tasks", task_id, "labels")
        # we use a PUT here so that if we fail in the middle, we can try again and replace the labels
        label_upload_response = requests.put(label_upload_url, headers=headers, json=label_upload_body)
        # make sure this post request doesn't fail silently
        label_upload_response.raise_for_status()

In [ ]:
class_map = get_class_map(hitl_project)
upload_labels(hitl_project["id"], class_map=class_map, **labels_with_task_ids)

### Putting it all together

The above steps are all separated out, but we can instead write a single function that runs through the entire workflow like so:

In [ ]:
def create_rv_label_project(source_project_id, iteration_number, rv_output_uri):
    hitl_project = clone_project(source_project_id, iteration_number)
    hitl_project_tasks = clone_tasks(source_project_id, hitl_project["id"])
    
    with open(rv_output_uri, "r") as inf:
        predictions_feature_collection = json.load(inf)
    labels_with_task_ids = associate_tasks(predictions_feature_collection, hitl_project_tasks)
    class_map = get_class_map(hitl_project)
    upload_labels(hitl_project["id"], class_map=class_map, **labels_with_task_ids)
    return hitl_project

In [ ]:
# You can skip this for now unless you want to create another project
create_rv_label_project(source_project_id, 1, rv_output_uri)

We'll return to this step later when we run through the loop again.

## From GroundWork to new training data

Before, we set a task status of `UNLABELED` when we created tasks, then `LABELED` when we uploaded labels to them. There's a more advanced status, `VALIDATED`, that indicates that a human has reviewed some labels and signed off on them. We can use the validation process in GroundWork to correct the predictions produced by Raster Vision.

The validation workflow uses GroundWork. After you're done with that, come back here.

...

...

...

Done validating? Great. Let's create some new training data.

To do that, we'll create a STAC export. STAC is short for the [spatio-temporal asset catalog specification](https://github.com/radiantearth/stac-spec), an open, extensible standard for describing geospatial data. GroundWork knows how to export, and Raster Vision knows how to train models from, STAC catalogs implementing the [label extension](https://github.com/stac-extensions/label). We can create a new export using the `/stac` endpoint:

In [ ]:
def create_validated_stac_export(campaign_id):
    export_post = {
        "name": "GW HITL Workshop export",
        "license": {"license": "proprietary"},
        "taskStatuses": ["VALIDATED"],
        "exportAssetType": None,
        "campaignId": campaign_id
    }
    resp = requests.post(f"{url_base}/api/stac", headers=headers, json=export_post)
    resp.raise_for_status()
    return resp.json()

We can wait for the export to complete within the notebook by checking its status and sleeping -- exports don't take very long so we won't have to sleep too often:

In [ ]:
def wait_for_export(export):
    export = requests.get(f"""{url_base}/api/stac/{export["id"]}""", headers=headers).json()
    if export["exportStatus"] != "EXPORTED":
        time.sleep(10)
        return wait_for_export(export)
    return export

Similarly to before, we can create a single function that creates and waits for the export, then prints the download URL:

In [ ]:
def export_campaign(campaign_id):
    export = create_validated_stac_export(campaign_id)
    completed = wait_for_export(export)
    return completed["downloadUrl"]

In [ ]:
campaign_id = hitl_project["campaignId"]
download_url = export_campaign(campaign_id)
download_url

We can download and unzip that export easily with bash:

In [ ]:
iter_num = 1

!mkdir -p export-data/iter-{iter_num}
!wget -O export-data/iter-{iter_num}/stac-export.zip "{download_url}"

### Inspect the STAC export

Let's see what's in the export and how Raster Vision is going to turn this into training data.

In [ ]:
# the downloaded location of the zip containing the STAC export we created before
stac_export_uri = f"./export-data/iter-{iter_num}/stac-export.zip"

In [ ]:
def preview_zipfile(path):
    with TemporaryDirectory() as tmp_dir:
        unzip(path, target_dir=tmp_dir)
        s = sd.seedir(path=tmp_dir, style='lines', indent=4, first='files', printout=False)
        root = s.split('\n')[0]
        s = s.replace(root, f'{path}/')
        print(s)

In [ ]:
preview_zipfile(stac_export_uri)

In [ ]:
def show_parsed_stac(project_infos):
    project_infos = deepcopy(project_infos)
    for info in project_infos:
        g = shape(info['aoi_geometry'])
        info['aoi_geometry']['coordinates'] = '[[...]]'
        info['image_bbox'] = info['image_bbox'].bounds
        info['label_bbox'] = info['label_bbox'].bounds
    print(pformat(project_infos))

`read_stac()` is a util function that Raster Vision uses to extract labels and other data from the STAC export.

In [ ]:
project_infos = read_stac(stac_export_uri, f'./tmp/inspect-export-{iter_num}')

In [ ]:
show_parsed_stac(project_infos)

If you want a closer look at the export contents, you can unzip it and then explore it using the notebook server's file browser. Start with `README.md`.

In [ ]:
!unzip -d export-data/iter-{iter_num}/ export-data/iter-{iter_num}/stac-export.zip

### What do the training labels look like?

As a sanity check, we can visualize the labels that we will actually be feeding into Raster Vision for training.

In [ ]:
def geojson_to_shape(path):
    with open(path, 'r') as f:
        features = json.load(f)['features']
    polygons = [shape(f['geometry']) for f in features]
    polygons = unary_union(polygons)
    return polygons

In [ ]:
def show_labels_combined(infos):
    fig = plt.figure(figsize=(12, 18))
    ax = plt.gca()
    extent = Polygon.from_bounds(*unary_union([info['label_bbox'] for info in infos]).bounds)
    ax.add_patch(MPolygon(np.array(extent.exterior), fill=None, hatch='/', alpha=.5))
    for info in infos:
        aoi_polygons = shape(info['aoi_geometry'])
        if isinstance(aoi_polygons, Polygon):
            aoi_polygons = MultiPolygon([aoi_polygons])
        label_polygons = geojson_to_shape(info['label_uri'])
        if isinstance(label_polygons, Polygon):
            label_polygons = MultiPolygon([label_polygons])
        for p in aoi_polygons:
            ax.add_patch(MPolygon(np.array(p.exterior), fc='white', ec='#777'))
        for p in label_polygons:
            ax.add_patch(MPolygon(np.array(p.exterior), fc='r', ec='r'))
    plt.axis('off')
    plt.autoscale()
    legend_patches = [
        Patch(fc='w', ec='k', hatch='/', alpha=.5, label='Unlabeled region'),
        Patch(fc='w', ec='#777', alpha=1, label='Labeled region'),
        Patch(color='r', alpha=1, label='Boat'),
    ]
    plt.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(.5, .985), fontsize='xx-large', frameon=False, ncol=3)
    fig.tight_layout(pad=0)
    plt.show()

In [ ]:
show_labels_combined(project_infos)

To get training chips, Raster Vision will randomly sample 300x300 (pixels) chips from the labeled regions only. If a chip happens to intersect with any boat polygons, it will be assigned the class `"boat"`, otherwise it will be considered a `"background"` chip. Raster Vision will train a classification model that classifies chips into one of two classes: `"boat"` and `"background"`.

## From new training data to new predictions

Now that we have new training data, we can train a new model based on our original model. We can load our original model from a file containing its weights. Again, this step starts with some configuration. We'll configure a few values that control how the Raster Vision training works:

- `MODEL_INIT_WEIGHTS_PATH`: where the pre-trained model weights live
- `OUTPUT_ROOT`: the directory that will serve as the base for next training runs
- `RV_CONFIG_FILE`: a python module containing a `get_config` function that returns a Raster Vision experiment config
- `stac_export_uri`: the downloaded location of the zip containing the STAC export we created before

In [ ]:
MODEL_INIT_WEIGHTS_PATH = "../init_weights.pth"
OUTPUT_ROOT = "./output"
RV_CONFIG_FILE = "./active_learning.py"

output_dir = f"{OUTPUT_ROOT}/iter_{iter_num}"
stac_export_uri = f"./export-data/iter-{iter_num}/stac-export.zip"

With those values configured, we can re-train our original model with the new training data we just exported.

### Training and making predictions with Raster Vision

Raster Vision allows setting up a reusable pipline for a typical machine learning workflow. The full pipeline looks like this:

<img src="img/rv_blog_pipeline.png" alt="RV pipelien" style="width: 800px"/>

Here, we will only be making use of the `train` and `predict` stages. The pipeline is highly configurable, but most of the configuration has already been taken care of and is defined in the file `RV_CONFIG_FILE`.

We can run Raster Vision using the command below. Its components can be understood as:
- `run inprocess`: runs Raster Vision locally with the configured commands and arguments
- `"{RV_CONFIG_FILE}"`: Raster Vision will run the `get_config()` function defined in this file to get the pipeline configuration.
- `train predict`: runs only the `train` and `predict` stages
- `-a <key> <value>`: these options get passed to the `get_config()` function in `RV_CONFIG_FILE` as keyword arguments

In [ ]:
!rastervision run inprocess "{RV_CONFIG_FILE}" \
    train predict bundle \
    -a output_dir "{output_dir}" \
    -a stac_export_uri "{stac_export_uri}" \
    -a init_weights "{MODEL_INIT_WEIGHTS_PATH}" \
    -a num_epochs "3" \
    -a chips_per_scene "100"

### Inspect predictions

Now we have some new predictions -- we can see the distribution of predicted scores for whether a chip contains a boat like so:

In [ ]:
with open(f"{output_dir}/predict/val-0.json", "r") as f:
    preds = json.load(f)

scores_boat = np.array([softmax(f['properties']['scores'])[1] for f in preds['features']])

fig = plt.figure(figsize=(10, 8))
plt.hist(scores_boat, bins=30, alpha=0.5, label='boat')
plt.legend()
plt.show()

What do we do with those predictions? Well, we have a nice little workflow for getting them into GroundWork, and then to correct them and turn them into new training data. A shorter version of the workflow can be seen below --

In [ ]:
# Make a new project

# increment this value each time you create a new project
iter_num = 2

rv_output_uri = f"./output/iter_{iter_num - 1}/predict/val-0.json"

next_project = create_rv_label_project(source_project_id, iter_num, rv_output_uri)

Then go validate some labels, then create a new export:

In [ ]:
next_training_url = export_campaign(next_project["campaignId"])
next_training_url

Finally, download the new training data, and re-run training:

In [ ]:
output_dir = f"{OUTPUT_ROOT}/iter_{iter_num}"
stac_export_uri = f"./export-data/iter-{iter_num}/stac-export.zip"

!mkdir -p export-data/iter-{iter_num}
!wget -O export-data/iter-{iter_num}/stac-export.zip "{next_training_url}"
!rastervision run inprocess "{RV_CONFIG_FILE}" \
    train predict bundle \
    -a output_dir "{output_dir}" \
    -a num_epochs "3" \
    -a chips_per_scene "100" \
    -a stac_export_uri "{stac_export_uri}" \
    -a init_weights "{MODEL_INIT_WEIGHTS_PATH}"

!rastervision predict \
  {output_dir}/bundle/model-bundle.zip \
  ../jacksonville.sub.tif \
  {output_dir}/predict/val-0.json

You can re-run the above three cells, incrementing `iter_num` each time, with the validation step in the middle as many times as you want to improve the model.